# 手动实现softmax

In [2]:
import torch
from IPython import display
from d2l import torch as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

100.0%
100.0%
100.0%
100.0%


In [3]:
num_inputs = 784
num_outputs = 10

W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)

In [4]:
X = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
X.sum(0, keepdim=True), X.sum(1, keepdim=True)

(tensor([[5., 7., 9.]]),
 tensor([[ 6.],
         [15.]]))

### 定义softmax函数

In [5]:
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim=True)
    return X_exp / partition  # 这里应用了广播机制

这里来试一下这个函数
生成一个形状为 (2 行，5 列) 的张量，形状如输出所示，元素服从均值为 0、标准差为 1的正态分布（随机数，可正可负）。
使用softmax函数后查看结果，所有元素非负，总和为1。

In [6]:
X = torch.normal(0, 1, (2, 5))
X_prob = softmax(X)
X_prob, X_prob.sum(1)

(tensor([[0.1424, 0.1812, 0.5674, 0.0610, 0.0479],
         [0.1703, 0.1624, 0.0974, 0.0353, 0.5345]]),
 tensor([1.0000, 1.0000]))

### 神经网络前向传播过程

reshape((-1, W.shape[0])) 中，-1 是 “自动计算维度” 的占位符：无论原始 X 是多少维（比如可能是图片的三维张量 [batch, height, width]），都会被重塑为 (样本数, 输入特征数) 的二维张量。

在经过reshape后X的形状为"batch_size * 784",因为输入为28*28=784个像素，W的形状为"input_samples * num_output",因此为 784 * 10, b的形状为10

计算后进行softmax的操作，得到每一个feature的概率

In [7]:
def net(X):
    return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b)

### 定义损失函数


首先y代表真实值，其中有两个样本，样本1输入类别0，样本2属于类别2
y_hat是预测值，形状为2*3，代表两个样本分别为3个类别的可能性
最后一行是 PyTorch 的高级索引操作，作用是：从y_hat中取出 “每个样本真实类别对应的预测概率”。

In [8]:
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y_hat[[0, 1], y]

tensor([0.1000, 0.5000])

接下来实现交叉熵

$ loss = - log(y_{true})$


range(len(y_hat))：生成样本索引（比如y_hat有 2 个样本，就生成[0, 1]），作用和前面的[0, 1]一致，但更通用（样本数变化时无需手动改）。

y_hat[range(len(y_hat)), y]：再次提取 “每个样本真实类别对应的预测概率”（和步骤 2 逻辑相同）。

In [9]:
def cross_entropy(y_hat, y):
    return - torch.log(y_hat[range(len(y_hat)), y])

cross_entropy(y_hat, y)

tensor([2.3026, 0.6931])